In [1]:
from colorlog import ColoredFormatter
import os
import requests
import logging
import sys
import redis

In [4]:
# ~~~ Configuration du Logger ~~~
def get_logger():
    global logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)

    formatter = ColoredFormatter(
        "%(log_color)s%(levelname)-8s%(reset)s %(message)s",
        log_colors={
            "DEBUG": "cyan",
            "INFO": "green",
            "WARNING": "yellow",
            "ERROR": "red",
            "CRITICAL": "red,bg_white",
        },
        reset=True,
        style="%",
    )

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    logging.basicConfig(level=logging.INFO, stream=sys.stdout)

    return logger

logger = get_logger()


In [5]:
# ~~~ Connection to Redis ~~~
def connect_to_redis():
    REDIS_HOST = os.getenv("REDIS_HOST","redisdb-master.default.svc.cluster.local")
    REDIS_HOST = "127.0.0.1"
    REDIS_PORT = os.getenv("REDIS_PORT","6379")
    REDIS_PASSWORD = os.getenv("REDIS_PASSWORD")
    REDIS_PASSWORD = ""
    logger.info(f"Connecting to Redis at host: {REDIS_HOST} on port: {REDIS_PORT}")
    r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, decode_responses=True, db=0)
    return r

In [6]:
r = connect_to_redis()
r.ping()

INFO     Connecting to Redis at host: 127.0.0.1 on port: 6379
INFO     Connecting to Redis at host: 127.0.0.1 on port: 6379


INFO:__main__:Connecting to Redis at host: 127.0.0.1 on port: 6379


True

In [34]:
redis_client = r

# ~~~ Redis Functions ~~~
def write_userdata_to_redis(dict_key, dict_mapping):
    logger.info(f"Writing to redis: {dict_key} {dict_mapping}")
    redis_client.hset(dict_key, mapping=dict_mapping)


def read_userdata_from_redis(dict_key):
    value = redis_client.hgetall(dict_key)
    logger.info(f"reading from redis: {value}")
    return value if value else None


In [35]:
write_userdata_to_redis("user1", {"name": "John", "age": 25, "city": "New York"})

INFO     Writing to redis: user1 {'name': 'John', 'age': 25, 'city': 'New York'}


INFO:__main__:Writing to redis: user1 {'name': 'John', 'age': 25, 'city': 'New York'}


In [37]:
coco = read_userdata_from_redis("user1")
coco

INFO     reading from redis: {'name': 'John', 'age': '25', 'city': 'New York'}


INFO:__main__:reading from redis: {'name': 'John', 'age': '25', 'city': 'New York'}


{'name': 'John', 'age': '25', 'city': 'New York'}